In [1]:
"""

Not function yet......

"""

###
###  Prepare data: extract, transform, load (ETL)
###
import h5py as h5
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
    
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

LABLE_WIDTH = 1
DATA_WIDTH  = 8192
X_FEATURE = 'x'  # Name of the input feature.

RATE = 8192
BATCH = 128
EPOCHS = 300

#FLAGS = tf.app.flags.FLAGS
#tf.app.flags.DEFINE_string("train_data_path", "/home/yongcai/chinese_fenci/train.txt", "training data dir")
#tf.app.flags.DEFINE_string("log_dir", "./logs", " the log dir")
#tf.app.flags.DEFINE_integer("max_sentence_len", 80, "max num of tokens per query")
#tf.app.flags.DEFINE_integer("shuffle_buffer_size", )

 
class GWInject():
    def _add_noise(self, tag, A):
        var = self.f[tag]
        NX = len(var)
        X = (A * var[:NX,:] + np.random.normal(0,1,(NX,self.srate))).astype(np.float32)
        Y = np.ones(NX)
        NN = NX
        X  = np.vstack( (X, np.random.normal(0,1,(NN,self.srate)) )  ).astype(np.float32)
        Y  = np.hstack( (Y, np.zeros(NN) )                          ).astype(np.float32).reshape(-1,1)
        
        if 1:
            plt.figure(figsize=(15,6))
            for i in range(len(X)):
                plt.subplot(4,5,i+1)
                plt.plot(X[i,:])
                plt.plot(A * var[i,:])
                #plt.title(" )
                if (i > 18): break
            plt.show()

        return X, Y
    def __init__(self, fname):
        self.fname = fname
        self.f = h5.File(fname, "r")
        self.srate = self.f.attrs.get('srate')

    def __exit__(self):
        self.f.close()
        
    def get_train_val_set(self, A=1.0):
        X, Y = self._add_noise('/train', A)
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, shuffle=True, random_state=None)
        return X_train, X_val, Y_train, Y_val
        
    def get_test_set(self, A=1.0):
        X, Y = self._add_noise('/test', A)
        return X,Y
        
    
###  TODO: generate the training set on-the-fly with lamdba mapping  ??    
#ds = WaveDS("white_h_fixed.h5", A=1)
#ds.plot()

#train_ds = ds.train.batch(BATCH)
#test_ds  = ds.test.batch(BATCH)




/home/lincy/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#### StoppingLossHook
#### LoggerHook
####
class StoppingLossHook(tf.train.SessionRunHook):
    def __init__(self, monitor, eps=0.01, patience=5, max=10000):
        self.monitor = monitor
        self.eps = eps
        self.patience = patience
        self.wait = 0
        self.max = max

    def begin(self):
        # Convert names to tensors if given
        graph = tf.get_default_graph()
        self.monitor = graph.as_graph_element(self.monitor)
        self.iter = 0
        if isinstance(self.monitor, tf.Operation):
            self.monitor = self.monitor.outputs[0]

    def before_run(self, run_context):  # pylint: disable=unused-argument
        return tf.train.SessionRunArgs(self.monitor)

    def after_run(self, run_context, run_values):
        self.iter += 1

        if self.iter>self.max: run_context.request_stop()  

        if run_values.results > self.eps:
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                run_context.request_stop()  
                
class LoggerHook(tf.train.SessionRunHook):
    def __init__(self, monitor, interval=100):
        self.monitor = monitor
        self.interval = interval

    def begin(self):
        self._step = -1
        self._start_time = time.time()

        # Convert names to tensors if given
        graph = tf.get_default_graph()
        self.monitor = graph.as_graph_element(self.monitor)
        if isinstance(self.monitor, tf.Operation):
            self.monitor = self.monitor.outputs[0]

    def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(self.monitor)

    def after_run(self, run_context, run_values):
        if self._step % self.interval == 0:

            res = run_values.results

            current_time = time.time()
            duration = current_time - self._start_time
            self._start_time = current_time

            # sample / second
            examples_per_sec = self.interval * BATCH / duration
            # time / batch
            sec_per_batch = float(duration / self.interval)
            format_str = ('Step: %4d, loss: %.2g  ( %.1f examples/sec; %.1f sec/batch)')
            print(format_str % (self._step, res, examples_per_sec, sec_per_batch))




In [3]:
###
### Prepare NN model
###

def convl(in_, F, K, D, S, PO, PS, act):
    args = {"padding":'valid', "activation":None, 
        "kernel_initializer":tf.truncated_normal_initializer(), 
        "bias_initializer":tf.zeros_initializer()     }
    out = tf.layers.conv1d( in_, filters=F, kernel_size=K, dilation_rate=D, strides=S, **args)
    out = tf.layers.max_pooling1d(out, pool_size=PO, strides=PS, padding='valid')
    return act(out)

def conv_model_hi(x_):
    feature = tf.reshape(x_, [-1, DATA_WIDTH,1])
    c1 = convl(feature, F= 64, K=16, D=1, S=1, PO=4, PS=4, act=tf.nn.relu)
    c2 = convl(c1,      F=128, K=16, D=2, S=1, PO=4, PS=4, act=tf.nn.relu)
    c3 = convl(c2,      F=256, K=16, D=2, S=1, PO=4, PS=4, act=tf.nn.relu)
    c4 = convl(c3,      F=512, K=32, D=2, S=1, PO=4, PS=4, act=tf.nn.relu)

    dim = c4.get_shape().as_list()
    fcnn = dim[1]*dim[2]
    l1 = tf.reshape(c4, [-1, fcnn])
    
    l2     = tf.layers.dense(l1, 128, activation=tf.nn.relu)
    l3     = tf.layers.dense(l2,  64, activation=tf.nn.relu)
    logits = tf.layers.dense(l3,   1, activation=None)
    return logits

def conv_model_low(x_):
    feature = tf.reshape(x_, [-1, DATA_WIDTH,1])

    o1 = convl(feature, F=16, K=16, D=1, S=1, PO=4, PS=4, act=tf.nn.relu)
    o2 = convl(o1,      F=32, K=8,  D=4, S=1, PO=4, PS=4, act=tf.nn.relu)
    o3 = convl(o2,      F=64, K=8,  D=4, S=1, PO=4, PS=4, act=tf.nn.relu)
    
    dim = o3.get_shape().as_list()
    fcnn = dim[1]*dim[2]
    l1 = tf.reshape(o3, [-1, fcnn])
    # Densely connected layer
    l2     = tf.layers.dense(l1, 64, activation=tf.nn.relu, name="dense_layer")
    logits = tf.layers.dense(l2,  1, activation=None, name="logit")
    return logits

    
def model(x_, labels, mode):

    #logits = conv_model_low(x_)
    logits = conv_model_hi(x_)
    
    # Compute predictions
    predict_prob = tf.sigmoid(logits, name="sigmoid_tensor")
    predict_op   = tf.round(predict_prob)  ## return largest index   ##tf.cast( tf.round(predict_prob), tf.int32 )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class': predict_op,
            'prob': predict_prob
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Create training op.
    loss_op = tf.losses.sigmoid_cross_entropy(logits=logits, multi_class_labels=labels)
    #print ("================ loss: ", mode, loss.get_shape())
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        #learning_rate = tf.train.exponential_decay(0.05, tf.train.get_global_step(), decay_steps=2000, decay_rate=0.95, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        #optimizer = tf.train.AdamOptimizer(1e-4)
        train_op  = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
        #logger_hook    = LoggerHook(monitor=loss,          interval=50)
        #logger_hook2   = LoggerHook(monitor=learning_rate, interval=100)
        stopping_hook = StoppingLossHook(monitor=loss, eps=1e-2, max=9999)
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op, training_hooks=[stopping_hook])

    # Compute evaluation metrics.
    eval_metric_ops = {
        #'accuracy':  tf.losses.sigmoid_cross_entropy(logits=logits, multi_class_labels=labels)
        'accuracy':    tf.metrics.accuracy(labels=labels, predictions=predict_op  ),
        'sensitivity': tf.metrics.recall(labels=labels, predictions=predict_op  )
    }
    return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [19]:
###
###  Construct TF graph
###
tf.reset_default_graph()
DIM = 8192
keep_prob = tf.placeholder(tf.float32)  ##  for dropout
bs        = tf.placeholder(tf.int64)    ##  for initalizer

x = tf.placeholder(tf.float32, [None,DIM])
y = tf.placeholder(tf.float32, [None,1])

train_ds = tf.data.Dataset.from_tensor_slices((x, y)).shuffle(buffer_size=10*bs).batch(bs).repeat()
test_ds  = tf.data.Dataset.from_tensor_slices((x, y)).batch(bs)

## Use two iterators
#train_iter = train_ds.make_initializable_iterator()  ## 
#test_iter  = test_ds.make_initializable_iterator()   ## no shuffle/repeat

## Use one common iterator
iter = tf.data.Iterator.from_structure(train_ds.output_types, train_ds.output_shapes)
features, labels = iter.get_next()

# create the initialisation operations
train_init_op = iter.make_initializer(train_ds)
test_init_op = iter.make_initializer(test_ds)

#graph_location = '/tmp/tf_tmp'
#merged = tf.summary.merge_all()
#print('Saving graph to: %s' % graph_location)

### check all variables
if 1:
    vars = 0
    for v in tf.global_variables():
        print (v)
        vars += np.prod(v.get_shape().as_list())
    print("Whole size: %.3f MB | Var # : %d" % (8*vars/(1024**2), len(tf.global_variables()) ) )

    vars = 0
    for v in tf.trainable_variables():
        print (v)
        vars += np.prod(v.get_shape().as_list())
    print("Model size: %.3f MB | Var # : %d" % (8*vars/(1024**2), len(tf.trainable_variables()) ) )

    vars = 0
    for v in tf.local_variables():
        print (v)
        vars += np.prod(v.get_shape().as_list())
    print("Local var size: %.3f Bytes | Var # : %d" % (8*vars, len(tf.local_variables()) ) )

[None, 119, 64]
Saving graph to: /tmp/tf_tmp
<tf.Variable 'conv1d/kernel:0' shape=(16, 1, 16) dtype=float32_ref>
<tf.Variable 'conv1d/bias:0' shape=(16,) dtype=float32_ref>
<tf.Variable 'conv1d_1/kernel:0' shape=(8, 16, 32) dtype=float32_ref>
<tf.Variable 'conv1d_1/bias:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv1d_2/kernel:0' shape=(8, 32, 64) dtype=float32_ref>
<tf.Variable 'conv1d_2/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'dense_layer/kernel:0' shape=(7616, 64) dtype=float32_ref>
<tf.Variable 'dense_layer/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'logit/kernel:0' shape=(64, 1) dtype=float32_ref>
<tf.Variable 'logit/bias:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'adam_optimizer/beta1_power:0' shape=() dtype=float32_ref>
<tf.Variable 'adam_optimizer/beta2_power:0' shape=() dtype=float32_ref>
<tf.Variable 'conv1d/kernel/Adam:0' shape=(16, 1, 16) dtype=float32_ref>
<tf.Variable 'conv1d/kernel/Adam_1:0' shape=(16, 1, 16) dtype=float32_ref>
<tf.Variable 

In [32]:
##
##  Use Numpy matrix
##
STEPS=100

with tf.Session() as sess:
    
    #train_writer = tf.summary.FileWriter(graph_location + '/train', sess.graph)
    #train_writer.add_graph(tf.get_default_graph())
    #test_writer  = tf.summary.FileWriter(graph_location + '/test')

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(train_init_op, feed_dict={ x:Xt, y:Yt, bs:BATCH})   ##, batch_size: BATCH
    
    for i in range(STEPS):
        optimizer.run()
        if i % 100 == 0:
            _, loss, acc = sess.run( [optimizer, loss_op, accuracy] )   ## 
            print('step: %5d, loss: %10.2g accuracy: %10.2g' % (i, loss, acc))

    # test model
    sess.run(test_init_op, feed_dict={ x: Xts, y: Yts, bs:BATCH })
    for i in range(int(len(Xts)/BATCH)+2):
        pre, loss, acc = sess.run([predict_prob, loss_op, accuracy])
        print("Test Loss: %.f Acc: %.3f" % (loss, acc) )
    

    
def train_input_fn():
    """
    return training set (x,y)
    """
    ds = tf.data.TFRecordDataset(FLAGS.train_dataset)
    train_dataset = train_dataset.map(parser)
    # num_epochs 为整个数据集的迭代次数
    train_dataset = train_dataset.repeat(FLAGS.num_epochs)
    train_dataset = train_dataset.batch(FLAGS.batch_size)
    train_iterator = train_dataset.make_one_shot_iterator()

    features, labels = train_iterator.get_next()
    return features, labels
    
    
def CNN(A):
    tf.logging.set_verbosity(tf.logging.INFO)   ### WARN
       
    GWdata = GWInject("white_h_fixed.h5")
    X_train, X_val, y_train, y_val = GWdata.get_train_val_set(A)
    X_test , Y_test                = GWdata.get_test_set(A)
    print("Test set size: %.3f MBytes" % (np.prod(np.shape(Xts))*4/1024**2 ) )
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = { X_FEATURE: X_train },  y = y_train,
        batch_size = BATCH,  ### step = N / Batch * epoches
        num_epochs = EPOCHS,  shuffle=True)
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = { X_FEATURE: X_test },   y= y_test,
        #batch_size = None,
        num_epochs=1,   shuffle=False)

    ########## CNN
    #try: os.remove('/tmp/tf_lin/checkpoint')  
    #except OSError: pass

    config = tf.contrib.learn.RunConfig(
        save_checkpoints_steps=10000000000,  ## at 1 and last
        save_checkpoints_secs=None,
        save_summary_steps=None,
        #gpu_memory_fraction=0.5,
        model_dir='/tmp/tf_lin'
    )
    classifier = tf.estimator.Estimator(model_fn=conv_model, config=config)
    
    classifier.train(input_fn=train_input_fn, steps=None)
    #classifier.export_savedmodel('/tmp/tf_lin/saved', serving_input_receiver_fn=train_input_fn, strip_default_attrs=True)
    
    
    scores = classifier.evaluate(input_fn=test_input_fn, steps=None)
    


    if 0:  
        ######### Linear classifier.
        feature_columns = [ tf.feature_column.numeric_column(X_FEATURE, shape=RATE) ]

        classifier = tf.estimator.LinearClassifier(feature_columns = feature_columns, n_classes=LABLE_WIDTH)
        classifier.train(input_fn=train_input_fn, steps=None)
        scores = classifier.evaluate(input_fn=test_input_fn, steps=None)
        print("+++++ LC: ", scores)

    
gloss = []
gacc  = []
gsen  = []
gstep = []
def main(unused):
    try: os.remove('/tmp/tf_gwda/checkpoint')  
    except OSError: pass

    #saver = tf.train.Saver()
    #snr = [0.075,0.05,0.025, 0.02, 0.01, 0.005, 0.003, 0.002, 0.001,  0.0001]
    snr = [1.0, 0.8, 0.6, 0.4, 0.3, 0.2,0.1, 0.05]
    for i in snr:
        l, a, s = CNN(i)
        gloss.append(l)
        gacc.append(a)
        gsen.append(s)
        gstep.append(tf.train.get_global_step())
    print("Loss  :", gloss)
    print("Acc   :", gacc)
    print("Recall:", gsen)
        
if __name__ == '__main__':
    tf.app.run()


(7740, 8192)
(2580, 8192)
(7740, 1)
(2580, 1)
Test set size: 315.375 MBytes
step:     0, loss:    5.8e+02 accuracy:       0.52
Test Loss: 56 Acc: 0.469
Test Loss: 88 Acc: 0.414
Test Loss: 55 Acc: 0.426
Test Loss: 74 Acc: 0.412
Test Loss: 84 Acc: 0.397
Test Loss: 70 Acc: 0.401
Test Loss: 61 Acc: 0.402
Test Loss: 69 Acc: 0.398
Test Loss: 55 Acc: 0.404
Test Loss: 75 Acc: 0.398
Test Loss: 67 Acc: 0.400
Test Loss: 71 Acc: 0.401
Test Loss: 64 Acc: 0.398
Test Loss: 60 Acc: 0.398
Test Loss: 65 Acc: 0.399
Test Loss: 63 Acc: 0.403
Test Loss: 59 Acc: 0.405
Test Loss: 72 Acc: 0.405
Test Loss: 64 Acc: 0.406
Test Loss: 57 Acc: 0.407
Test Loss: 68 Acc: 0.406
Test Loss: 72 Acc: 0.403
Test Loss: 68 Acc: 0.401
Test Loss: 69 Acc: 0.401
Test Loss: 83 Acc: 0.396
Test Loss: 54 Acc: 0.398
Test Loss: 55 Acc: 0.402
Test Loss: 67 Acc: 0.403
Test Loss: 68 Acc: 0.402
Test Loss: 67 Acc: 0.401
Test Loss: 66 Acc: 0.402
Test Loss: 67 Acc: 0.402
Test Loss: 73 Acc: 0.401
Test Loss: 68 Acc: 0.400
Test Loss: 68 Acc: 0.40

OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,8192], [?,1]], output_types=[DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]
	 [[Node: IteratorGetNext/_25 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_85_IteratorGetNext", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op u'IteratorGetNext', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in l